In [26]:
import numpy as np
import pandas as pd
import random
import math
import os
import scipy.io
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import transformers
%matplotlib inline
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from math import sqrt
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [27]:
time = pd.read_csv(r"C:\Users\HP\Desktop\scaai\Bearing 1_1 time_domain.csv")
frequency = pd.read_csv(r"C:\Users\HP\Desktop\scaai\Bearing 1_1 frequency_domain.csv")


In [28]:
time.drop("Unnamed: 0", axis = 1, inplace = True)
time.drop("Signal", axis = 1, inplace = True)
frequency.drop("Unnamed: 0", axis = 1, inplace = True)

In [33]:
Y = time["Time"]

In [34]:
X = pd.concat([time.drop("Time", axis = 1),frequency.drop("Time", axis = 1)], axis = 1)
X = pd.DataFrame(X)

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle=False)


In [13]:
'''''
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import r2_score


clf = RandomForestRegressor(n_estimators=100, random_state=42)

sfs = SequentialFeatureSelector(clf, n_features_to_select=20, direction='forward', scoring='r2', cv=2)

sfs.fit(X_train, y_train)


selected_features = X.columns[sfs.get_support()]

X_train_selected = sfs.transform(X_train)
X_test_selected = sfs.transform(X_test)

clf.fit(X_train_selected, y_train)

y_pred = clf.predict(X_test_selected)

r2 = r2_score(y_test, y_pred)
print(f"Selected Features: {selected_features}")
print(f"Accuracy on Test Set: {r2}")
'''''

Selected Features: Index(['crest_factor_Y', 'variance_X', 'standard_deviations_Y', 'p2p_X',
       'p2p_Y', 'Entropy_X', 'Entropy_Y', 'Skewness_X', 'wave_factor_X',
       'peak_factor_Y', 'skewness_coefficent_X', 'freq_mean_fx',
       'freq_peak_fx', 'spectral_entropy_fx', 'spectral_skew_fx', 'psd_pg_fx',
       'spectral_entropy_fy', 'spectral_flatness_fy', 'ZCR_fy', 'psd_pg_fy'],
      dtype='object')
Accuracy on Test Set: 0.96930828736409


In [36]:
cols = ['crest_factor_Y', 'variance_X', 'standard_deviations_Y', 'p2p_X',
       'p2p_Y', 'Entropy_X', 'Entropy_Y', 'Skewness_X', 'wave_factor_X',
       'peak_factor_Y', 'skewness_coefficent_X',   'freq_mean_fx',
       'freq_peak_fx', 'spectral_entropy_fx', 'spectral_skew_fx', 'psd_pg_fx',
       'spectral_entropy_fy', 'spectral_flatness_fy', 'ZCR_fy', 'psd_pg_fy']  

In [38]:
scores = []
for col in cols:
    X_train_selected = np.array(X_train[col]).reshape(-1,1)
    X_test_selected = np.array(X_test[col]).reshape(-1,1)
    scaler = StandardScaler()
    X_train_standardized = scaler.fit_transform(X_train_selected).tolist()
    X_test_standardized = scaler.transform(X_test_selected).tolist()
    from torch.utils.data import DataLoader, TensorDataset
    SEQUENCE_SIZE = 50

    def to_sequences(seq_size, obs):
        x = []
        y = []
        for i in range(len(obs) - seq_size):
            window = obs[i:(i + seq_size)]
            after_window = obs[i + seq_size]
            x.append(window)
            y.append(after_window)
        return torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), torch.tensor(y, dtype=torch.float32).view(-1, 1)

    x_train, y_train = to_sequences(SEQUENCE_SIZE, X_train_standardized)
    x_test, y_test = to_sequences(SEQUENCE_SIZE, X_test_standardized)

    # Setup data loaders for batch
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(x_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    class PositionalEncoding(nn.Module):
        def __init__(self, d_model, dropout=0.1, max_len=5000):
            super(PositionalEncoding, self).__init__()
            self.dropout = nn.Dropout(p=dropout)

            pe = torch.zeros(max_len, d_model)
            position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
            div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
            pe[:, 0::2] = torch.sin(position * div_term)
            pe[:, 1::2] = torch.cos(position * div_term)
            pe = pe.unsqueeze(0).transpose(0, 1)
            self.register_buffer('pe', pe)

        def forward(self, x):
            x = x + self.pe[:x.size(0), :]
            return self.dropout(x)
    class TransformerModel(nn.Module):
        def __init__(self, input_dim=1, d_model=256, nhead=4, num_layers=8, dropout=0.1):
            super(TransformerModel, self).__init__()

            self.encoder = nn.Linear(input_dim, d_model)
            self.pos_encoder = PositionalEncoding(d_model, dropout)
            encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=512, dropout=dropout, activation='relu')
            self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
            self.decoder = nn.Linear(d_model, 1)

        def forward(self, x):
            x = self.encoder(x)
            x = self.pos_encoder(x)
            x = self.transformer_encoder(x)
            x = self.decoder(x[:, -1, :])
            return x
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TransformerModel().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

    epochs = 1000
    early_stop_count = 0
    min_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        val_losses = []
        with torch.no_grad():
            for batch in test_loader:
                x_batch, y_batch = batch
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                val_losses.append(loss.item())

        val_loss = np.mean(val_losses)
        scheduler.step(val_loss)

        if val_loss < min_val_loss:
            min_val_loss = val_loss
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= 10:
            print("Early stopping!")
            break
        print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")
    from sklearn.metrics import r2_score

    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch = x_batch.to(device)
            outputs = model(x_batch)
            predictions.extend(outputs.squeeze().tolist())

    rmse = np.sqrt(np.mean((scaler.inverse_transform(np.array(predictions).reshape(-1, 1)) - scaler.inverse_transform(y_test.numpy().reshape(-1, 1)))**2))
    print(f"RMSE for {col}: {rmse:.4f}")

    r2 = r2_score(scaler.inverse_transform(y_test.numpy().reshape(-1, 1)), scaler.inverse_transform(np.array(predictions).reshape(-1, 1)))
    print(f"R2 Score for {col}: {r2:.4f}")
    scores.append([col, rmse, r2])

    


Epoch 1/1000, Validation Loss: 1.3324
Epoch 2/1000, Validation Loss: 0.9225
Epoch 3/1000, Validation Loss: 0.9395
Epoch 4/1000, Validation Loss: 0.8615
Epoch 5/1000, Validation Loss: 1.0078
Epoch 6/1000, Validation Loss: 1.0203
Epoch 7/1000, Validation Loss: 1.1656
Epoch 00008: reducing learning rate of group 0 to 5.0000e-04.
Epoch 8/1000, Validation Loss: 1.0059
Epoch 9/1000, Validation Loss: 0.9561
Epoch 10/1000, Validation Loss: 1.0635
Epoch 11/1000, Validation Loss: 0.9978
Epoch 00012: reducing learning rate of group 0 to 2.5000e-04.
Epoch 12/1000, Validation Loss: 0.8728
Epoch 13/1000, Validation Loss: 0.9232
Early stopping!
RMSE for crest_factor_Y: 1.5658
R2 Score for crest_factor_Y: -0.2020
Epoch 1/1000, Validation Loss: 1744.3049
Epoch 2/1000, Validation Loss: 1710.4954
Epoch 3/1000, Validation Loss: 1712.9743
Epoch 4/1000, Validation Loss: 1713.7687
Epoch 5/1000, Validation Loss: 1712.3329
Epoch 00006: reducing learning rate of group 0 to 5.0000e-04.
Epoch 6/1000, Validation L